https://blog.csdn.net/qq_31367861/article/details/111145816?spm=5176.12282029.0.0.24ac6e12RdK2dO 

采用其方法仅对其中的两万条数据进行复现

比赛链接：https://tianchi.aliyun.com/competition/entrance/531830/information

In [1]:
# coding: utf-8
import multiprocessing
from collections import Counter
import xgboost as xgb
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm import tqdm
from sklearn.model_selection import KFold
import gc
from sklearn import preprocessing
from scipy.stats import entropy
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.metrics import roc_auc_score, roc_curve
import datetime
import time
from itertools import product


In [2]:
DATA_PATH = '/Users/senlinlidewo/pyprogram/天池竞赛-零基础金融风控/数据/'

In [8]:
def load_dataset(DATA_PATH):
    train_label = pd.read_csv(DATA_PATH + 'train.csv')['isDefault']
    train = pd.read_csv(DATA_PATH + 'train.csv')
    test = pd.read_csv(DATA_PATH + 'testA.csv')
    feats = [f for f in train.columns if f not in ['isDefault']]
    # train = train[feats]
    test = test[feats]
    print('train.shape', train.shape)
    print('test.shape', test.shape)

    return train_label, train, test

In [9]:
train_label, train, test = load_dataset(DATA_PATH=DATA_PATH)

train.shape (800000, 47)
test.shape (200000, 46)


train1 = pd.read_csv('/Users/senlinlidewo/pyprogram/天池竞赛-零基础金融风控/数据/train.csv')
test1 = pd.read_csv('/Users/senlinlidewo/pyprogram/天池竞赛-零基础金融风控/数据/testA.csv')

In [10]:
train = train.sample(20000)
test = test.sample(10000)
train.shape
#这个sample每次运行都会发生改变

(20000, 47)

In [12]:
# 如何确认哪些列含有缺失值？
def missing_percentage(df):
    """This function takes a DataFrame(df) as input and returns two columns, total missing values and total missing values percentage"""
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2)
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])
missing_value_percent = missing_percentage(train)
missing_value_percent

,Total,Percent
n11,1759,8.80
employmentLength,1134,5.67
n7,1047,5.24
n1,1047,5.24
n2,1047,5.24
n3,1047,5.24
n5,1047,5.24
n6,1047,5.24
n14,1047,5.24
n8,1047,5.24


In [13]:
cols = missing_value_percent.index[missing_value_percent.Percent>0]
list(cols)

['n11',
 'employmentLength',
 'n7',
 'n1',
 'n2',
 'n3',
 'n5',
 'n6',
 'n14',
 'n8',
 'n9',
 'n12',
 'n13',
 'n0',
 'n4',
 'n10',
 'revolUtil',
 'pubRecBankruptcies',
 'dti']

In [14]:
for col in cols:
    train[col].fillna(r'\N', inplace=True)
cols = [f for f in cols if f not in ['employmentLength']]
for col in cols:
    train[col].replace({r'\N': -999}, inplace=True)
    train[col] = train[col]
#先把所有的空缺值填为/N，然后employmentLength列保持不变，其他的空缺值由/n变为-999

In [15]:
missing_percentage(train)

,Total,Percent
postCode,1,0.0
id,0,0.0
revolBal,0,0.0
totalAcc,0,0.0
initialListStatus,0,0.0
applicationType,0,0.0
earliesCreditLine,0,0.0
title,0,0.0
policyCode,0,0.0
n0,0,0.0


In [14]:
#我想知道employlength里面的数据种类
def percent_value_counts(df, feature):
    """This function takes in a dataframe and a column and finds the percentage of the value_counts"""
    percent = pd.DataFrame(round(df.loc[:,feature].value_counts(dropna=False, normalize=True)*100,2))
    ## creating a df with th
    total = pd.DataFrame(df.loc[:,feature].value_counts(dropna=False))
    ## concating percent and total dataframe

    total.columns = ["Total"]
    percent.columns = ['Percent']
    return pd.concat([total, percent], axis = 1)
percent_value_counts(train,'employmentLength')

,Total,Percent
12.0,6617,33.08
2.0,1868,9.34
0.5,1569,7.85
3.0,1527,7.64
1.0,1310,6.55
5.0,1242,6.21
4.0,1220,6.10
-999.0,1155,5.78
6.0,925,4.62
7.0,915,4.58


In [16]:
def employmentLength_deal(x):
    if x == r'\N':
        result = -999
    elif x == -999:
        result = -999
    elif x == '-999':
        result = -999
    elif x == '< 1 year':
        result = 0.5
    elif x == '10+ years':
        result = 12
    else:
        result = int(x.split(' ')[0][0])
    # print(result)
    return result
train['employmentLength'] = train['employmentLength'].apply(lambda x: employmentLength_deal(x))

In [9]:
# 生成分类变量list
train_object = list(train.select_dtypes(include = ['object']))
train_object
# 这样有一定的风险，比如postcode是多分类变量而其类型是float。 

['grade', 'subGrade', 'employmentLength', 'issueDate', 'earliesCreditLine']

In [10]:
# 无序变量容易被默认为numberic    'postCode', 'purpose', 'regionCode', 'homeOwnership', 'employmentTitle','title'
# 有序多分类变量   'grade', 'subGrade'
# object但是是连续变量 'issueDate', 'earliesCreditLine' 'employmentLength'

In [10]:
# Meancode 编码无序变量 
#平均编码类定义
class MeanEncoder:
    def __init__(self, categorical_features, n_splits=5, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode

        :param n_splits: the number of splits used in mean encoding

        :param target_type: str, 'regression' or 'classification'

        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """

        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}

        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None

        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))

    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()

        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()

        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg([('mean','mean'), ('beta','size')])
        col_avg_y['beta'] = prior_weight_func(col_avg_y['beta'])
        col_avg_y[nf_name] = col_avg_y['beta'] * prior + (1 - col_avg_y['beta']) * col_avg_y['mean']
        col_avg_y.drop(['beta', 'mean'], axis=1, inplace=True)

        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values

        return nf_train, nf_test, prior, col_avg_y

    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)

        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target,
                        self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(y, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None,
                        self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new

    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()

        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits

        return X_new

In [18]:
class_list = ['postCode', 'purpose', 'regionCode', 'homeOwnership', 'employmentTitle','title']
MeanEnocodeFeature = class_list  # 声明需要平均数编码的特征
ME = MeanEncoder(MeanEnocodeFeature, target_type='classification')
train = ME.transform(train) 
# 声明平均数编码的类
##这个MeanEncoder是怎么调用的？调用前后数据集发生了什么变化呢？

In [19]:
#对有序分类进行处理

def gradeTrans(x):
    dict = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
    result = dict[x]
    return result


def subGradeTrans(x):
    dict = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
    result = dict[x[0]]
    result = result * 5 + int(x[1])
    return result
    
train['grade'] = train['grade'].apply(lambda x: gradeTrans(x))
train['subGrade'] = train['subGrade'].apply(lambda x: subGradeTrans(x))


import datetime


cols = ['issueDate', 'earliesCreditLine']
for col in cols:
    print(train[col])


for col in cols:
    train[col] = pd.to_datetime(train[col])
    train[col] = train[col].map(lambda x: x.year)
#dataframe apply函数
    

In [20]:
def earliesCreditLine_month_deal(x):
    x = x.split('-')[0]
    # print(x)
    dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
            'Nov': 11, 'Dec': 12}
    result = dict[x]
    return result

In [22]:
data = train

In [34]:
data['avg_income'] = data['annualIncome'] / data['employmentLength']
data['total_income'] = data['annualIncome'] * data['employmentLength']
data['avg_loanAmnt'] = data['loanAmnt'] / data['term']
data['mean_interestRate'] = data['interestRate'] / data['term']
data['all_installment'] = data['installment'] * data['term']

data['rest_money_rate'] = data['avg_loanAmnt'] / (data['annualIncome'] + 0.1)  # 287个收入为0
data['rest_money'] = data['annualIncome'] - data['avg_loanAmnt']

data['closeAcc'] = data['totalAcc'] - data['openAcc']

data['rest_pubRec'] = data['pubRec'] - data['pubRecBankruptcies']

data['rest_Revol'] = data['loanAmnt'] - data['revolBal']



In [35]:
data['earliesCreditLine_year'] = data['earliesCreditLine'].apply(lambda x: 2020 - (int(x.split('-')[-1])))
data['earliesCreditLine_month'] = data['earliesCreditLine'].apply(lambda x: earliesCreditLine_month_deal(x))
data['earliesCreditLine_Allmonth'] = data['earliesCreditLine_year'] * 12 - data['earliesCreditLine_month']


In [37]:
##交叉特征
from tqdm import tqdm
col_cat = ['subGrade', 'grade', 'employmentLength', 'term', 'homeOwnership', 'postCode', 'regionCode','employmentTitle','title']
col_num = ['dti', 'revolBal','revolUtil', 'interestRate', 'loanAmnt', 'installment', 'annualIncome', 'n14',
             'n2', 'n6', 'n9', 'n5', 'n8']
             
# 定义离散型特征和连续型特征交叉特征统计函数
def cross_cat_num(df, num_col, cat_col):
    for f1 in tqdm(cat_col):
        g = df.groupby(f1, as_index=False)
        for f2 in tqdm(num_col):
            feat = g[f2].agg({
                '{}_{}_max'.format(f1, f2): 'max', '{}_{}_min'.format(f1, f2): 'min',
                '{}_{}_median'.format(f1, f2): 'median',
            })
            df = df.merge(feat, on=f1, how='left')
    return (df)

In [38]:
data = cross_cat_num(data, col_num, col_cat) 


100%|██████████| 9/9 [00:06<00:00,  1.50it/s]


In [39]:
#暴力特征
def myEntro(x):
    """
        calculate shanno ent of x
    """
    x = np.array(x)
    x_value_list = set([x[i] for i in range(x.shape[0])])
    ent = 0.0
    for x_value in x_value_list:
        p = float(x[x == x_value].shape[0]) / x.shape[0]
        logp = np.log2(p)
        ent -= p * logp
    #     print(x_value,p,logp)
    # print(ent)
    return ent

#求均方根
def myRms(records):
    records = list(records)
    """
    均方根值 反映的是有效值而不是平均值
    """
    return np.math.sqrt(sum([x ** 2 for x in records]) / len(records))


def myMode(x):
    return np.mean(pd.Series.mode(x))
    

def myQ25(x):
    return x.quantile(0.25)
    
def myQ75(x):
    return x.quantile(0.75)

def myQ10(x):
    return x.quantile(0.1)
    
def myQ90(x):
    return x.quantile(0.9)
    

def myRange(x):
    return pd.Series.max(x) - pd.Series.min(x)

n_feat = ['n0', 'n1', 'n2', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n12', 'n13', 'n14', ]
nameList = ['min', 'max', 'sum', 'mean', 'median', 'skew', 'std', 'mode', 'range', 'Q25','Q75']
statList = ['min', 'max', 'sum', 'mean', 'median', 'skew', 'std', myMode, myRange, myQ25, myQ75]

for i in range(len(nameList)):
	data['n_feat_{}'.format(nameList[i])] = data[n_feat].agg(statList[i], axis=1)
print('n特征处理后：', data.shape)


n特征处理后： (20000, 425)


In [ ]:
help(folds.split)

In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_data,train_target,test_size = 0.2,random_state=6666)
len(X_train)

NameError: name 'train_data' is not defined

In [46]:
def xgb_model(train, target, test, k):

    feats = [f for f in train.columns if f not in ['id', 'isDefault']]
    feaNum = len(feats)
    print('参与训练的特征数目:', len(feats))
#     seeds = [6666,2020]
    seeds = [2020]
    output_preds = 0
    xgb_oof_probs = np.zeros(train.shape[0])

    for seed in seeds:
        folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
        oof_probs = np.zeros(train.shape[0])

        offline_score = []
        feature_importance_df = pd.DataFrame()
        params = {'booster': 'gbtree',
                  'objective': 'binary:logistic',
                  'eval_metric': 'auc',
                  'min_child_weight': 5,
                  'max_depth': 8,
                  'eta': 0.01,
                  # 'scale_pos_weight': 0.2,
                  'seed': seed,
                  'nthread': -1,
                  'tree_method': 'gpu_hist'
                  }
        for i, (train_index, test_index) in enumerate(folds.split(train, target)):
            
            train_y, test_y = target[train_index], target[test_index]
            train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
            train_matrix = xgb.DMatrix(train_X, label=train_y, missing=np.nan)
            valid_matrix = xgb.DMatrix(test_X, label=test_y, missing=np.nan)
            test_matrix = xgb.DMatrix(test[feats], missing=np.nan)
            watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]
            model = xgb.train(params, train_matrix, num_boost_round=30000, evals=watchlist, verbose_eval=100,
                              early_stopping_rounds=600)
            val_pred = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)
            train_pred = model.predict(train_matrix, ntree_limit=model.best_ntree_limit)
            xgb_oof_probs[test_index] += val_pred / len(seeds)
            # oof_probs[test_index] += val_pred
            test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit)

            # 绘制roc曲线
            train_auc_value, valid_auc_value = plotroc(train_y, train_pred, test_y, val_pred)
            print('train_auc:{},valid_auc{}'.format(train_auc_value, valid_auc_value))
            offline_score.append(valid_auc_value)
            print(offline_score)
            output_preds += test_pred / k / len(seeds)
         
            fold_importance_df = pd.DataFrame()
            fold_importance_df["Feature"] = model.get_fscore().keys()
            fold_importance_df["importance"] = model.get_fscore().values()
            fold_importance_df["fold"] = i + 1

            feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)



        print('all_auc:', roc_auc_score(target.values, oof_probs))
        print('OOF-MEAN-AUC:%.6f, OOF-STD-AUC:%.6f' % (np.mean(offline_score), np.std(offline_score)))
        feature_sorted = feature_importance_df.groupby(['Feature'])['importance'].mean().sort_values(ascending=False)
        feature_sorted.to_csv('feature_importance.csv')
        top_features = feature_sorted.index
        print(feature_importance_df.groupby(['Feature'])['importance'].mean().sort_values(ascending=False).head(50))
    return output_preds, xgb_oof_probs, np.mean(offline_score), feaNum



In [14]:
help(xgb.train)

Help on function train in module xgboost.training:

train(params, dtrain, num_boost_round=10, evals=(), obj=None, feval=None, maximize=None, early_stopping_rounds=None, evals_result=None, verbose_eval=True, xgb_model=None, callbacks=None)
    Train a booster with given parameters.
    
    Parameters
    ----------
    params : dict
        Booster params.
    dtrain : DMatrix
        Data to be trained.
    num_boost_round: int
        Number of boosting iterations.
    evals: list of pairs (DMatrix, string)
        List of validation sets for which metrics will evaluated during training.
        Validation metrics will help us track the performance of the model.
    obj : function
        Customized objective function.
    feval : function
        Customized evaluation function.
    maximize : bool
        Whether to maximize feval.
    early_stopping_rounds: int
        Activates early stopping. Validation metric needs to improve at least once in
        every **early_stopping_round

In [58]:
train = X_train
target = y_train
test = X_test


In [49]:
xgb_preds, xgb_oof, xgb_score, feaNum = xgb_model(train=train, target=target, test=test, k=5)

参与训练的特征数目: 423


ValueError: Found input variables with inconsistent numbers of samples: [20000, 800000]

In [42]:
train = data[~data['isDefault'].isnull()].copy()
target = 

In [48]:
test = data[~data['isDefault'].isnull()].sample(20000)

In [44]:
target = train_label

In [ ]:
#以下是源代码

03-11
读取数据...
train.shape (800000, 47)
test.shape (200000, 46)
初始拼接后： (1000000, 47)
n特征处理后： (1000000, 53)
count编码后： (1000000, 61)
1data.shape (1000000, 53)
2_data.shape (1000000, 57)
预处理完毕 (1000000, 58)
开始特征工程...
data.shape (1000000, 74)
开始模型训练...
num0:mean_encode train.shape (800000, 84) (800000, 84)


  0%|          | 0/5 [00:00<?, ?it/s]

num1:target_encode train.shape (800000, 89) (800000, 89)


100%|██████████| 5/5 [02:38<00:00, 31.71s/it]


num2:target_encode train.shape (800000, 109) (800000, 109)
输入数据维度： (800000, 104) (800000, 104)
Current num of features: 102
[0]	train-auc:0.70731	eval-auc:0.69928
[100]	train-auc:0.73562	eval-auc:0.72400
[200]	train-auc:0.74134	eval-auc:0.72716
[300]	train-auc:0.74719	eval-auc:0.73013
[400]	train-auc:0.75263	eval-auc:0.73257
[500]	train-auc:0.75746	eval-auc:0.73435
[600]	train-auc:0.76159	eval-auc:0.73560
[700]	train-auc:0.76537	eval-auc:0.73655
[800]	train-auc:0.76876	eval-auc:0.73716
[900]	train-auc:0.77206	eval-auc:0.73772
[1000]	train-auc:0.77526	eval-auc:0.73813
[1100]	train-auc:0.77821	eval-auc:0.73852
[1200]	train-auc:0.78088	eval-auc:0.73885
[1300]	train-auc:0.78346	eval-auc:0.73909
[1400]	train-auc:0.78613	eval-auc:0.73926
[1500]	train-auc:0.78882	eval-auc:0.73947
[1600]	train-auc:0.79134	eval-auc:0.73964
[1700]	train-auc:0.79373	eval-auc:0.73979
[1800]	train-auc:0.79612	eval-auc:0.73995
[1900]	train-auc:0.79847	eval-auc:0.74005
[2000]	train-auc:0.80064	eval-auc:0.74013
[2100]

KeyboardInterrupt: 